In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from collections import Counter
from imblearn.under_sampling import TomekLinks
from collections import Counter
import string

## Import data

In [25]:
path='data/edition_translation/'

In [133]:
#data import
train_data_ecco=pd.read_csv(path+'ecco_monolingual_train.csv')
train_data_ecco['ecco_full_title']=train_data_ecco['ecco_full_title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
test_data_ecco=pd.read_csv(path+'ecco_monolingual_test.csv')
test_data_ecco['ecco_full_title']=test_data_ecco['ecco_full_title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
train_data_caa=pd.read_csv(path+'caa_monolingual_train.csv')
train_data_caa['title']=train_data_caa['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
test_data_caa=pd.read_csv(path+'caa_monolingual_test.csv')
test_data_caa['title']=test_data_caa['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
balanced_train_data_caa=pd.read_csv(path+'caa_monolingual_balanced_train.csv')
balanced_train_data_caa['title']=balanced_train_data_caa['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
balanced_test_data_caa=pd.read_csv(path+'caa_monolingual_balanced_test.csv')
balanced_test_data_caa['title']=balanced_test_data_caa['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
combined_train_data=pd.read_csv(path+'combined_monolingual_train.csv')
combined_train_data['title']=combined_train_data['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()
combined_test_data=pd.read_csv(path+'combined_monolingual_test.csv')
combined_test_data['title']=combined_test_data['title'].str.translate(str.maketrans("", "", string.punctuation)).str.lower()

In [27]:
combined=pd.concat([combined_train_data,combined_test_data])

In [28]:
combined[(combined['monolingual']==1)]

,Unnamed: 0,level_0,index,title,id,monolingual,multilingual,monolingual_translations,source
0,3421,4115,1127,nicolai vernvlæi ottocarvs bohemiæ rex sev reb...,9911760850101488,1,0,0,caa
1,5891,8689,2234,pars secunda tractatvs de jure devolutionis in...,9992391009101488,1,0,0,caa
2,961,985,1879,joannis ludovici vives colloquia alioqui latin...,998537850101488,1,0,0,caa
3,4966,6781,679,oratio fvnebris in obitvm matthiæ hovii archi...,9978909880101488,1,0,0,caa
4,5479,7790,2740,ivstificatio sev defensio censvræ facvltatis s...,9935323420101488,1,0,0,caa
...,...,...,...,...,...,...,...,...,...
1041,4350,5628,2497,philippus iv,9983928540101488,1,0,0,caa
1042,3126,3698,98,benedicti xiv commentarius de sacrosancto mis...,992119560101488,1,0,0,caa
1043,323,328,2827,de lipsii latinitate vt ipsimet antiquarii ant...,9911797690101488,1,0,0,caa
1044,3509,4235,1811,æsops fables with his life and morals and rema...,63801300,1,0,1,ecco


In [29]:
# Concatenate test and train data
combined_test_train = pd.concat([combined_test_data, combined_train_data])

# Tokenize the text and remove punctuation
def preprocess_text(text):
    # Remove punctuation using string.punctuation
    cleaned_text = text.translate(str.maketrans("", "", string.punctuation))
    return [word for word in cleaned_text.lower().split()]

tokens_cleaned = combined_test_train['title'].apply(preprocess_text)

# Calculate word frequencies
word_counts = Counter(word for sublist in tokens_cleaned for word in sublist)

# Initialize presence_counts dictionary
presence_counts = {}

# Check if the word is not in word_counts and calculate presence count
for sublist in tokens_cleaned:
    for word in sublist:
        if word not in word_counts:
            presence_counts[word] = len(combined_test_train[combined_test_train['title'].str.lower().apply(lambda x: word in x)])

# Filter based on frequency and proportion
n_titles = len(combined_test_train)
common_vocabulary = [word for word, count in word_counts.items() if 1 < count <= 0.8 * n_titles and word not in presence_counts]

# Train on ECCO, test on ECCO for original vs translation

In [30]:
train_data=train_data_ecco

In [31]:
id_field="estc_id"
source_field="ecco_full_title"
target_field="monolingual_translations"
x_train=train_data[['estc_id', 'ecco_full_title']]
y_train=train_data[target_field]

In [32]:
# Define a pipeline to search for the best combination of TFidf vectorizer and logistic regression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(vocabulary=common_vocabulary, lowercase=True)),
    ('clf', LinearSVC())
])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3)],
    'clf__penalty': ['l1', 'l2', 'elasticnet', None],
    'clf__class_weight': [None, 'balanced'],
}
search = GridSearchCV(pipeline, param_grid)
search.fit(x_train[source_field], y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.962):
{'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'tfidf__ngram_range': (1, 1)}


C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_classes.p

In [33]:
clf__class_weight=None
clf__penalty= 'l2' 
tfidf__ngram_range= (1, 1)

### I now do the prediction based on the best hyperparameters

In [34]:
text_transformer = TfidfVectorizer(vocabulary=common_vocabulary, ngram_range=tfidf__ngram_range, lowercase=True, max_features=150000)
clf= LinearSVC(class_weight=clf__class_weight, penalty=clf__penalty)

In [35]:
x_test=test_data_ecco[[id_field, source_field]]
y_test=test_data_ecco[target_field]

In [36]:
X_train_text = text_transformer.fit_transform(x_train[source_field])
X_test_text = text_transformer.transform(x_test[source_field])

In [37]:
clf.fit(X_train_text, y_train)
test_preds = clf.predict(X_test_text)

In [38]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94       128
           1       0.95      0.98      0.97       225

    accuracy                           0.96       353
   macro avg       0.96      0.95      0.95       353
weighted avg       0.96      0.96      0.96       353



In [39]:
accuracy_score(y_test, test_preds)

0.9575070821529745

## I now do the prediction on caa

In [40]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"

In [41]:
x_test=test_data_caa[[id_field, source_field]]
y_test=test_data_caa[target_field]
x_test_balanced=balanced_test_data_caa[[id_field, source_field]]
y_test_balanced=balanced_test_data_caa[target_field]

In [42]:
X_test_caa = text_transformer.transform(x_test[source_field])
X_balanced_test_caa=text_transformer.transform(x_test_balanced[source_field])

In [43]:
y_preds_caa = clf.predict(X_test_caa)
y_preds_balanced=clf.predict(X_balanced_test_caa)

In [44]:
print(classification_report(y_test, y_preds_caa))

              precision    recall  f1-score   support

           0       0.99      0.65      0.79       663
           1       0.10      0.84      0.18        31

    accuracy                           0.66       694
   macro avg       0.54      0.75      0.48       694
weighted avg       0.95      0.66      0.76       694



In [45]:
print(classification_report(y_test_balanced, y_preds_balanced))

              precision    recall  f1-score   support

           0       0.93      0.60      0.73        72
           1       0.51      0.91      0.65        33

    accuracy                           0.70       105
   macro avg       0.72      0.75      0.69       105
weighted avg       0.80      0.70      0.70       105



In [46]:
accuracy_score(y_test, y_preds_caa)

0.659942363112392

In [47]:
accuracy_score(y_test_balanced, y_preds_balanced)

0.6952380952380952

### I test on the combined dataset

In [48]:
id_field="id"
source_field="title"
target_field="monolingual_translations"

In [49]:
x_test=combined_test_data[[id_field, source_field]]
y_test=combined_test_data[target_field]

In [50]:
X_test = text_transformer.transform(x_test[source_field])

In [51]:
y_preds = clf.predict(X_test)

In [52]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.98      0.69      0.81       800
           1       0.49      0.96      0.65       246

    accuracy                           0.75      1046
   macro avg       0.74      0.83      0.73      1046
weighted avg       0.87      0.75      0.77      1046



In [53]:
accuracy_score(y_test, y_preds)

0.7523900573613767

### I train and predict on CAA

In [54]:
train_data=train_data_caa

In [55]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"
x_train=train_data[[id_field, source_field]]
y_train=train_data[target_field]

In [56]:
# Define a pipeline to search for the best combination of TFidf vectorizer and logistic regression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(vocabulary=common_vocabulary, lowercase=True)),
    ('clf', LinearSVC())
])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3)],
    'clf__penalty': ['l1', 'l2', 'elasticnet', None],
    'clf__class_weight': [None, 'balanced'],
}
search = GridSearchCV(pipeline, param_grid)
search.fit(x_train[source_field], y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.959):
{'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'tfidf__ngram_range': (1, 1)}


C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_classes.p

In [57]:
clf__class_weight='balanced'
clf__penalty= 'l2' 
tfidf__ngram_range= (1, 1)

## I now do the prediction based on the best hyperparameters

In [58]:
text_transformer = TfidfVectorizer(vocabulary=common_vocabulary, ngram_range=tfidf__ngram_range, lowercase=True, max_features=150000)
clf= LinearSVC(class_weight=clf__class_weight, penalty=clf__penalty)

In [59]:
x_test=test_data_caa[[id_field, source_field]]
y_test=test_data_caa[target_field]

In [60]:
X_train_text = text_transformer.fit_transform(x_train[source_field])
X_test_text = text_transformer.transform(x_test[source_field])
clf.fit(X_train_text, y_train)

LinearSVC(class_weight='balanced')

In [61]:
x_test=test_data_caa[[id_field, source_field]]
y_test=test_data_caa[target_field]
x_test_balanced=balanced_test_data_caa[[id_field, source_field]]
y_test_balanced=balanced_test_data_caa[target_field]

In [62]:
X_test_caa = text_transformer.transform(x_test[source_field])
X_balanced_test_caa=text_transformer.transform(x_test_balanced[source_field])

In [63]:
y_preds_caa = clf.predict(X_test_caa)
y_preds_balanced=clf.predict(X_balanced_test_caa)

In [64]:
print(classification_report(y_test, y_preds_caa))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       663
           1       0.55      0.52      0.53        31

    accuracy                           0.96       694
   macro avg       0.76      0.75      0.76       694
weighted avg       0.96      0.96      0.96       694



In [65]:
print(classification_report(y_test_balanced, y_preds_balanced))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96        72
           1       0.97      0.85      0.90        33

    accuracy                           0.94       105
   macro avg       0.95      0.92      0.93       105
weighted avg       0.94      0.94      0.94       105



In [66]:
accuracy_score(y_test, y_preds_caa)

0.9596541786743515

In [67]:
accuracy_score(y_test_balanced, y_preds_balanced)

0.9428571428571428

## I test on ECCO

In [68]:
id_field="estc_id"
source_field="ecco_full_title"
target_field="monolingual_translations"

In [69]:
x_test=test_data_ecco[[id_field, source_field]]
y_test=test_data_ecco[target_field]

In [70]:
X_test_text = text_transformer.transform(x_test[source_field])

In [71]:
test_preds=clf.predict(X_test_text)

In [72]:
test_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,

In [73]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.39      0.98      0.56       128
           1       0.91      0.13      0.23       225

    accuracy                           0.44       353
   macro avg       0.65      0.55      0.40       353
weighted avg       0.72      0.44      0.35       353



In [74]:
accuracy_score(y_test, test_preds)

0.43909348441926344

## I test on the combined dataset

In [75]:
id_field="id"
source_field="title"
target_field="monolingual_translations"

In [76]:
x_test=combined_test_data[[id_field, source_field]]
y_test=combined_test_data[target_field]

In [77]:
X_test = text_transformer.transform(x_test[source_field])

In [78]:
y_preds = clf.predict(X_test)

In [79]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       800
           1       0.90      0.26      0.40       246

    accuracy                           0.82      1046
   macro avg       0.86      0.63      0.65      1046
weighted avg       0.83      0.82      0.78      1046



In [80]:
accuracy_score(y_test, y_preds)

0.8193116634799236

# I train on both and test on each one singularly

In [81]:
train_data=combined_train_data

In [82]:
id_field="id"
source_field="title"
target_field="monolingual_translations"

In [83]:
x_train=train_data[[id_field,source_field]]
y_train=train_data[target_field]

In [84]:
# Define a pipeline to search for the best combination of TFidf vectorizer and logistic regression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(vocabulary=common_vocabulary, lowercase=True)),
    ('clf', LinearSVC())
])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3)],
    'clf__penalty': ['l1', 'l2', 'elasticnet', None],
    'clf__class_weight': [None, 'balanced'],
}
search = GridSearchCV(pipeline, param_grid)
search.fit(x_train[source_field], y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.954):
{'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'tfidf__ngram_range': (1, 1)}


C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_classes.p

In [85]:
clf__class_weight='balanced'
clf__penalty= 'l2' 
tfidf__ngram_range= (1, 1)

## I now do the prediction based on the best hyperparameters

In [86]:
text_transformer = TfidfVectorizer(vocabulary=common_vocabulary, ngram_range=tfidf__ngram_range, lowercase=True, max_features=150000)
clf= LinearSVC(class_weight=clf__class_weight, penalty=clf__penalty)

In [87]:
x_test=combined_test_data[[id_field, source_field]]
y_test=combined_test_data[[target_field, 'source']]

In [88]:
X_train_text = text_transformer.fit_transform(x_train[source_field])
X_test_text = text_transformer.transform(x_test[source_field])

In [89]:
clf.fit(X_train_text, y_train)
test_preds = clf.predict(X_test_text)

In [90]:
print(classification_report(y_test[target_field], test_preds))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       800
           1       0.93      0.89      0.91       246

    accuracy                           0.96      1046
   macro avg       0.95      0.93      0.94      1046
weighted avg       0.96      0.96      0.96      1046



In [91]:
accuracy_score(y_test[target_field], test_preds)

0.9579349904397706

## I test on ECCO

In [92]:
id_field="estc_id"
source_field="ecco_full_title"
target_field="monolingual_translations"

In [93]:
x_test=test_data_ecco[[id_field, source_field]]
y_test=test_data_ecco[target_field]

In [94]:
X_test_text = text_transformer.transform(x_test[source_field])
test_preds=clf.predict(X_test_text)

In [95]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95       128
           1       0.96      0.99      0.97       225

    accuracy                           0.96       353
   macro avg       0.97      0.95      0.96       353
weighted avg       0.96      0.96      0.96       353



In [96]:
accuracy_score(y_test, test_preds)

0.9631728045325779

## I test on unbalanced caa

In [97]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"

In [98]:
x_test=test_data_caa[[id_field, source_field]]
y_test=test_data_caa[target_field]

In [99]:
X_test_text = text_transformer.transform(x_test[source_field])
test_preds=clf.predict(X_test_text)

In [100]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       663
           1       0.93      0.90      0.92        31

    accuracy                           0.99       694
   macro avg       0.96      0.95      0.96       694
weighted avg       0.99      0.99      0.99       694



In [101]:
accuracy_score(y_test, test_preds)

0.9927953890489913

## I test on balanced CAA

In [102]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"

In [103]:
x_test=balanced_test_data_caa[[id_field, source_field]]
y_test=balanced_test_data_caa[target_field]

In [104]:
X_test_text = text_transformer.transform(x_test[source_field])

In [105]:
test_preds=clf.predict(X_test_text)

In [106]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99        72
           1       0.97      0.97      0.97        33

    accuracy                           0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



In [107]:
accuracy_score(y_test, test_preds)

0.9809523809523809

## I now redo the pipeline for CAA by sampling the data for more balanced classes

In [108]:
train_data=balanced_train_data_caa

In [109]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"

In [110]:
x_train=train_data[[id_field, source_field]]
y_train=train_data[target_field]

In [111]:
# Define a pipeline to search for the best combination of TFidf vectorizer and logistic regression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(vocabulary=common_vocabulary, lowercase=True)),
    ('clf', LinearSVC())
])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3)],
    'clf__penalty': ['l1', 'l2', 'elasticnet', None],
    'clf__class_weight': [None, 'balanced'],
}
search = GridSearchCV(pipeline, param_grid)
search.fit(x_train[source_field], y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.848):
{'clf__class_weight': None, 'clf__penalty': 'l2', 'tfidf__ngram_range': (1, 1)}


C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\u0140177\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_classes.p

In [112]:
clf__class_weight=None
clf__penalty= 'l2' 
tfidf__ngram_range= (1, 1)

## I now do the prediction based on the best hyperparameters

In [113]:
text_transformer = TfidfVectorizer(vocabulary=common_vocabulary, ngram_range=tfidf__ngram_range, lowercase=True, max_features=150000)
clf= LinearSVC(class_weight=clf__class_weight, penalty=clf__penalty)

In [114]:
x_test=balanced_test_data_caa[[id_field, source_field]]
y_test=balanced_test_data_caa[target_field]

In [115]:
X_train_text = text_transformer.fit_transform(x_train[source_field])
X_test_text = text_transformer.transform(x_test[source_field])

In [116]:
clf.fit(X_train_text, y_train)

LinearSVC()

In [117]:
y_preds = clf.predict(X_test_text)

In [118]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90        72
           1       0.78      0.76      0.77        33

    accuracy                           0.86       105
   macro avg       0.84      0.83      0.83       105
weighted avg       0.86      0.86      0.86       105



In [119]:
accuracy_score(y_test, y_preds)

0.8571428571428571

## I test on unbalanced caa

In [120]:
id_field="RecordID"
source_field="title"
target_field="monolingual_translations"

In [121]:
x_test=test_data_caa[[id_field, source_field]]
y_test=test_data_caa[target_field]

In [122]:
X_test_text = text_transformer.transform(x_test[source_field])

In [123]:
test_preds=clf.predict(X_test_text)

In [124]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97       663
           1       0.43      0.97      0.60        31

    accuracy                           0.94       694
   macro avg       0.72      0.95      0.78       694
weighted avg       0.97      0.94      0.95       694



In [125]:
accuracy_score(y_test, test_preds)

0.9423631123919308

## I test on ECCO

In [126]:
id_field="estc_id"
source_field="ecco_full_title"
target_field="monolingual_translations"

In [127]:
x_test=test_data_ecco[[id_field, source_field]]
y_test=test_data_ecco[target_field]

In [128]:
X_test_text = text_transformer.transform(x_test[source_field])

In [129]:
test_preds=clf.predict(X_test_text)

In [130]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.60      0.90      0.72       128
           1       0.92      0.66      0.77       225

    accuracy                           0.75       353
   macro avg       0.76      0.78      0.74       353
weighted avg       0.80      0.75      0.75       353



In [131]:
accuracy_score(y_test, test_preds)

0.7450424929178471